<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/pl_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-10jww83_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-10jww83_
  Resolved https://github.com/huggingface/transformers to commit dc68a39c8111217683bf49a4912d0c9018bab33d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [4]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [5]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "pl", split="test")
#column_name = "audio, normalized_text"

In [6]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("large")

In [7]:
model = whisper.load_model("large")

In [8]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [9]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="pl")
    return result["text"]

In [10]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [12]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [13]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)

    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1
    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.25, Precision: 0.75, Recall: 0.75, F1: 0.75
Processed 2 files, WER: 0.08, Precision: 0.9166666666666666, Recall: 0.9565217391304348, F1: 0.9361702127659574
Processed 3 files, WER: 0.36, Precision: 0.64, Recall: 0.6666666666666666, F1: 0.6530612244897959
Processed 4 files, WER: 0.3, Precision: 0.6666666666666666, Recall: 0.6666666666666666, F1: 0.6666666666666666
Processed 5 files, WER: 0.1875, Precision: 0.8666666666666667, Recall: 0.8125, F1: 0.8387096774193549
Processed 6 files, WER: 0.2857142857142857, Precision: 0.7368421052631579, Recall: 0.7, F1: 0.717948717948718
Processed 7 files, WER: 0.34782608695652173, Precision: 0.6818181818181818, Recall: 0.6818181818181818, F1: 0.6818181818181818
Processed 8 files, WER: 0.1, Precision: 0.9, Recall: 0.9, F1: 0.9
Processed 9 files, WER: 0.25, Precision: 0.75, Recall: 0.75, F1: 0.75
Processed 10 files, WER: 0.34210526315789475, Precision: 0.7631578947368421, Recall: 0.7631578947368421, F1: 0.7631578947368421
Proces